Feature engineering for model

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [2]:
# Load EDDMapS data
report = pd.read_csv("CS4824CapstoneData/EDDMapS_clean.csv", encoding='latin-1')
fips_df = pd.read_csv("CS4824CapstoneData/fips_clean.csv", encoding='latin-1')
counties = gpd.read_file("CS4824CapstoneData/tl_2025_us_county/tl_2025_us_county.shp")
urban_rural_df = pd.read_csv("CS4824CapstoneData/urban_rural_clean.csv")
state_lookup = pd.read_csv("CS4824CapstoneData/state_lookup_clean.csv")
tree_df = pd.read_csv("CS4824CapstoneData/treeOfHeavenDistributionData.csv")


print(f"Report shape: {report.shape}")
print(f"FIPS shape: {fips_df.shape}")
print(f"Counties shapefile shape: {counties.shape}")
print(f"Urban/Rural shape: {urban_rural_df.shape}")
print(f"State lookup shape: {state_lookup.shape}")
print(f"Tree of Heaven shape: {tree_df.shape}")
print(tree_df.columns)

Report shape: (26132, 8)
FIPS shape: (3143, 5)
Counties shapefile shape: (3235, 19)
Urban/Rural shape: (3144, 7)
State lookup shape: (51, 9)
Tree of Heaven shape: (846, 6)
Index(['Symbol', 'Country', 'State', 'State FIP', 'County', 'County FIP'], dtype='object')


In [3]:
fips_df.head()

,fips,name,state,County,State
0,1001,Autauga County,AL,autauga,al
1,1003,Baldwin County,AL,baldwin,al
2,1005,Barbour County,AL,barbour,al
3,1007,Bibb County,AL,bibb,al
4,1009,Blount County,AL,blount,al


In [4]:
# normalize strings

def normalize_county_name(name):
    if pd.isna(name):
        return ""
    name = str(name).lower().strip()
    for suffix in [' county', ' parish', ' borough', ' municipality', ' census area', ' city']:
        if name.endswith(suffix):
            name = name.replace(suffix, '')
    return name.strip()

#
report["County"] = report["County"].astype(str).str.lower().str.strip()
report["State"] = report["State"].astype(str).str.lower().str.strip()
report['County_norm'] = report['County'].apply(normalize_county_name)
report['State_norm'] = report['State'].str.lower().str.strip()

fips_df["County"] = fips_df["County"].astype(str).str.lower().str.strip()
fips_df["State"] = fips_df["State"].astype(str).str.lower().str.strip()
fips_df['County_norm'] = fips_df['County'].apply(normalize_county_name)
fips_df['State_norm'] = fips_df['State'].str.lower().str.strip()

urban_rural_df["State"] = urban_rural_df["State"].astype(str).str.lower().str.strip()
urban_rural_df["County_name"] = urban_rural_df["County_name"].astype(str).str.lower().str.strip()
urban_rural_df['County_norm'] = urban_rural_df['County_name'].apply(normalize_county_name)

state_lookup["Name"] = state_lookup["Name"].astype(str).str.lower().str.strip()
state_lookup["Abbrev"] = state_lookup["Abbrev"].astype(str).str.lower().str.strip()

tree_df["County"] = tree_df["County"].astype(str).str.lower().str.strip()
tree_df["State"] = tree_df["State"].astype(str).str.lower().str.strip()
tree_df['County_norm'] = tree_df['County'].apply(normalize_county_name)
tree_df['State_norm'] = tree_df['State'].str.lower().str.strip()

print("✓ All string columns normalized to lowercase")



STEP 1: NORMALIZING STRING COLUMNS
✓ All string columns normalized to lowercase


In [5]:
fips_df.head()

,fips,name,state,County,State,County_norm,State_norm
0,1001,Autauga County,AL,autauga,al,autauga,al
1,1003,Baldwin County,AL,baldwin,al,baldwin,al
2,1005,Barbour County,AL,barbour,al,barbour,al
3,1007,Bibb County,AL,bibb,al,bibb,al
4,1009,Blount County,AL,blount,al,blount,al


In [6]:

# ============================================================================
# STEP 2: STANDARDIZE FIPS CODES (5 digits with leading zeros)
# ============================================================================
print("\n" + "=" * 80)
print("STEP 2: STANDARDIZING FIPS CODES")
print("=" * 80)

# Pad FIPS codes to 5 digits
fips_df['fips'] = fips_df['fips'].astype(str).str.zfill(5)
urban_rural_df['FIPS code'] = urban_rural_df['FIPS code'].astype(str).str.zfill(5)

# Also handle the counties shapefile FIPS if it exists
if 'GEOID' in counties.columns:
    counties['FIPS'] = counties['GEOID'].astype(str).str.zfill(5)
elif 'FIPS' in counties.columns:
    counties['FIPS'] = counties['FIPS'].astype(str).str.zfill(5)

print(f"Sample FIPS codes from fips_df: {fips_df['fips'].head(3).tolist()}")
print(f"Sample FIPS codes from urban_rural_df: {urban_rural_df['FIPS code'].head(3).tolist()}")



STEP 2: STANDARDIZING FIPS CODES
Sample FIPS codes from fips_df: ['01001', '01003', '01005']
Sample FIPS codes from urban_rural_df: ['02013', '02016', '02020']


In [7]:
fips_df.head()

,fips,name,state,County,State,County_norm,State_norm
0,01001,Autauga County,AL,autauga,al,autauga,al
1,01003,Baldwin County,AL,baldwin,al,baldwin,al
2,01005,Barbour County,AL,barbour,al,barbour,al
3,01007,Bibb County,AL,bibb,al,bibb,al
4,01009,Blount County,AL,blount,al,blount,al


In [8]:
urban_rural_df.head()

,FullGeoName,FIPS code,State,County_name,2023 Code,urban_code,urban_description,County_norm
0,"AK, Aleutians East Borough",02013,ak,aleutians east borough,6 - Noncore,6,Noncore,aleutians east
1,"AK, Aleutians West Ca",02016,ak,aleutians west census area,6 - Noncore,6,Noncore,aleutians west
2,"AK, Anchorage Muny",02020,ak,anchorage municipality,3 - Medium metro,3,Medium metro,anchorage
3,"AK, Bethel Ca",02050,ak,bethel census area,6 - Noncore,6,Noncore,bethel
4,"AK, Bristol Bay Borough",02060,ak,bristol bay borough,6 - Noncore,6,Noncore,bristol bay


In [9]:
tree_df.head()

,Symbol,Country,State,State FIP,County,County FIP,County_norm,State_norm
0,AIAL,United States,alabama,1,nan,NaN,nan,alabama
1,AIAL,United States,alabama,1,autauga,1.0,autauga,alabama
2,AIAL,United States,alabama,1,bullock,11.0,bullock,alabama
3,AIAL,United States,alabama,1,butler,13.0,butler,alabama
4,AIAL,United States,alabama,1,calhoun,15.0,calhoun,alabama


In [10]:
report.head()

,objectid,ObsDate,year,Latitude,Longitude,County,State,Status_encoded,County_norm,State_norm
0,4999651,2017-08-10,2017,40.49447,-75.48765,lehigh,pennsylvania,1.0,lehigh,pennsylvania
1,7629151,2018-07-05,2018,40.39368,-76.03501,berks,pennsylvania,1.0,berks,pennsylvania
2,7629152,2018-07-05,2018,40.42861,-76.11261,berks,pennsylvania,1.0,berks,pennsylvania
3,7638053,2018-07-30,2018,40.48981,-76.10119,berks,pennsylvania,1.0,berks,pennsylvania
4,7811998,2018-09-21,2018,40.37313,-76.05336,berks,pennsylvania,1.0,berks,pennsylvania


In [11]:

# ============================================================================
# STEP 3: FILTER TO CONTINENTAL US ONLY (48 states + DC)
# ============================================================================
print("\n" + "=" * 80)
print("STEP 3: FILTERING TO CONTINENTAL US")
print("=" * 80)

# Define continental US states - abbreviated (exclude AK, HI, and territories)
continental_states_abbrev = [
    'al', 'az', 'ar', 'ca', 'co', 'ct', 'de', 'dc', 'fl', 'ga', 
    'id', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 
    'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nv', 'nh', 'nj', 'nm', 
    'ny', 'nc', 'nd', 'oh', 'ok', 'or', 'pa', 'ri', 'sc', 'sd', 
    'tn', 'tx', 'ut', 'vt', 'va', 'wa', 'wv', 'wi', 'wy'
]

# Define continental US states - full names (for tree_df and report)
continental_states_full = [
    'alabama', 'arizona', 'arkansas', 'california', 'colorado', 'connecticut', 
    'delaware', 'district of columbia', 'florida', 'georgia', 'idaho', 
    'illinois', 'indiana', 'iowa', 'kansas', 'kentucky', 'louisiana', 
    'maine', 'maryland', 'massachusetts', 'michigan', 'minnesota', 
    'mississippi', 'missouri', 'montana', 'nebraska', 'nevada', 
    'new hampshire', 'new jersey', 'new mexico', 'new york', 'north carolina', 
    'north dakota', 'ohio', 'oklahoma', 'oregon', 'pennsylvania', 
    'rhode island', 'south carolina', 'south dakota', 'tennessee', 'texas', 
    'utah', 'vermont', 'virginia', 'washington', 'west virginia', 
    'wisconsin', 'wyoming'
]

print(f"Continental US states: {len(continental_states_abbrev)} states + DC")

# Filter dataframes - use appropriate state list for each
fips_df = fips_df[fips_df['State_norm'].isin(continental_states_abbrev)].copy()
urban_rural_df = urban_rural_df[urban_rural_df['State'].isin(continental_states_abbrev)].copy()
tree_df = tree_df[tree_df['State_norm'].isin(continental_states_full)].copy()
report = report[report['State_norm'].isin(continental_states_full)].copy()

print(f"✓ FIPS after filter: {fips_df.shape[0]} counties")
print(f"✓ Urban/Rural after filter: {urban_rural_df.shape[0]} counties")
print(f"✓ Tree of Heaven after filter: {tree_df.shape[0]} records")
print(f"✓ Reports after filter: {report.shape[0]} reports")



STEP 3: FILTERING TO CONTINENTAL US
Continental US states: 49 states + DC
✓ FIPS after filter: 3109 counties
✓ Urban/Rural after filter: 3109 counties
✓ Tree of Heaven after filter: 843 records
✓ Reports after filter: 26072 reports


In [12]:
fips_df.head()

,fips,name,state,County,State,County_norm,State_norm
0,01001,Autauga County,AL,autauga,al,autauga,al
1,01003,Baldwin County,AL,baldwin,al,baldwin,al
2,01005,Barbour County,AL,barbour,al,barbour,al
3,01007,Bibb County,AL,bibb,al,bibb,al
4,01009,Blount County,AL,blount,al,blount,al


In [13]:
# ============================================================================
# STEP 4: CREATE BASE COUNTY DATAFRAME
# ============================================================================
print("\n" + "=" * 80)
print("STEP 4: CREATING BASE COUNTY DATAFRAME")
print("=" * 80)

# Start with fips_df as the base (all counties in continental US)
base_counties = fips_df[['fips', 'County_norm', 'State_norm', 'state']].copy()
base_counties.rename(columns={'state': 'state_abbrev'}, inplace=True)

print(f"Base counties dataframe: {base_counties.shape[0]} counties")
print(base_counties.head())



STEP 4: CREATING BASE COUNTY DATAFRAME
Base counties dataframe: 3109 counties
    fips County_norm State_norm state_abbrev
0  01001     autauga         al           AL
1  01003     baldwin         al           AL
2  01005     barbour         al           AL
3  01007        bibb         al           AL
4  01009      blount         al           AL


In [14]:
# ============================================================================
# STEP 5: ADD URBAN CODE
# ============================================================================
print("\n" + "=" * 80)
print("STEP 5: ADDING URBAN CODE")
print("=" * 80)

# Merge urban/rural data by FIPS code
urban_rural_subset = urban_rural_df[['FIPS code', 'urban_code']].copy()
urban_rural_subset.rename(columns={'FIPS code': 'fips'}, inplace=True)

base_counties = base_counties.merge(urban_rural_subset, on='fips', how='left')

print(f"Counties with urban_code: {base_counties['urban_code'].notna().sum()}")
print(f"Counties missing urban_code: {base_counties['urban_code'].isna().sum()}")
print(f"Urban code distribution:\n{base_counties['urban_code'].value_counts().sort_index()}")




STEP 5: ADDING URBAN CODE
Counties with urban_code: 3099
Counties missing urban_code: 10
Urban code distribution:
urban_code
1.0      66
2.0     367
3.0     387
4.0     353
5.0     652
6.0    1274
Name: count, dtype: int64


In [15]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code
0,01001,autauga,al,AL,3.0
1,01003,baldwin,al,AL,4.0
2,01005,barbour,al,AL,5.0
3,01007,bibb,al,AL,2.0
4,01009,blount,al,AL,2.0


In [16]:
state_lookup.head()

,Name,Abbrev,Population,Pop2000,IncomePerCapita,PercentCollegeGrad,MedianRent,CommuteTime,LandArea
0,alabama,al,4779736,4447100,32419,21.5,435,23.7,50645.33
1,alaska,ak,710231,626932,40042,26.5,857,17.9,570640.95
2,arizona,az,6392017,5130632,32833,25.7,709,24.9,113594.08
3,arkansas,ar,2915918,2673400,30177,18.9,425,21.0,52035.48
4,california,ca,37253956,33871648,41805,29.7,1002,27.0,155779.22


In [17]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code
0,01001,autauga,al,AL,3.0
1,01003,baldwin,al,AL,4.0
2,01005,barbour,al,AL,5.0
3,01007,bibb,al,AL,2.0
4,01009,blount,al,AL,2.0


In [18]:
tree_df.head()

,Symbol,Country,State,State FIP,County,County FIP,County_norm,State_norm
0,AIAL,United States,alabama,1,nan,NaN,nan,alabama
1,AIAL,United States,alabama,1,autauga,1.0,autauga,alabama
2,AIAL,United States,alabama,1,bullock,11.0,bullock,alabama
3,AIAL,United States,alabama,1,butler,13.0,butler,alabama
4,AIAL,United States,alabama,1,calhoun,15.0,calhoun,alabama


In [19]:
# ============================================================================
# STEP 6: ADD TREE OF HEAVEN PRESENCE
# ============================================================================
print("\n" + "=" * 80)
print("STEP 6: ADDING TREE OF HEAVEN PRESENCE")
print("=" * 80)

# Create tree of heaven presence indicator
# Any county appearing in tree_df has presence = 1
tree_presence = tree_df[['County_norm', 'State_norm']].drop_duplicates()
tree_presence['tree_of_heaven'] = 1

print(f"Unique counties with Tree of Heaven: {len(tree_presence)}")
print(f"Sample tree_presence data (full state names):")
print(tree_presence.head())

# Convert state full names to abbreviations for matching with base_counties
# Create a mapping from state_lookup (full name -> abbreviation)
state_full_to_abbrev = dict(zip(state_lookup['Name'], state_lookup['Abbrev']))

print(f"\nConverting state names to abbreviations...")
tree_presence['State_norm'] = tree_presence['State_norm'].map(state_full_to_abbrev)
print('tree_presence.head(): ',tree_presence.head())
# Check if any states failed to map
#if tree_presence['state_abbrev'].isna().any():
#    print(f"⚠️ WARNING: {tree_presence['state_abbrev'].isna().sum()} records failed to map state names")
#    print("States that failed:")
#    print(tree_presence[tree_presence['state_abbrev'].isna()]['State_norm'].unique())

#print(f"\nSample after state conversion:")
#print(tree_presence[['County_norm', 'State_norm', 'state_abbrev', 'tree_of_heaven']].head())



STEP 6: ADDING TREE OF HEAVEN PRESENCE
Unique counties with Tree of Heaven: 843
Sample tree_presence data (full state names):
  County_norm State_norm  tree_of_heaven
0         nan    alabama               1
1     autauga    alabama               1
2     bullock    alabama               1
3      butler    alabama               1
4     calhoun    alabama               1

Converting state names to abbreviations...
tree_presence.head():    County_norm State_norm  tree_of_heaven
0         nan         al               1
1     autauga         al               1
2     bullock         al               1
3      butler         al               1
4     calhoun         al               1


In [20]:
tree_presence.head()

,County_norm,State_norm,tree_of_heaven
0,nan,al,1
1,autauga,al,1
2,bullock,al,1
3,butler,al,1
4,calhoun,al,1


In [21]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code
0,01001,autauga,al,AL,3.0
1,01003,baldwin,al,AL,4.0
2,01005,barbour,al,AL,5.0
3,01007,bibb,al,AL,2.0
4,01009,blount,al,AL,2.0


In [22]:

# Merge with base counties using County_norm and state_abbrev
base_counties = base_counties.merge(
    tree_presence[['County_norm', 'State_norm', 'tree_of_heaven']], 
    on=['County_norm', 'State_norm'], 
    how='left'
)

# Fill NaN with 0 (no tree of heaven presence)
base_counties['tree_of_heaven'] = base_counties['tree_of_heaven'].fillna(0).astype(int)

print(f"\n✓ Counties with Tree of Heaven: {(base_counties['tree_of_heaven'] == 1).sum()}")
print(f"✓ Counties without Tree of Heaven: {(base_counties['tree_of_heaven'] == 0).sum()}")
print(f"Sample counties with Tree of Heaven:")
print(base_counties[base_counties['tree_of_heaven'] == 1][['fips', 'County_norm', 'state_abbrev']].head())




✓ Counties with Tree of Heaven: 807
✓ Counties without Tree of Heaven: 2302
Sample counties with Tree of Heaven:
     fips County_norm state_abbrev
0   01001     autauga           AL
5   01011     bullock           AL
6   01013      butler           AL
7   01015     calhoun           AL
10  01021     chilton           AL


In [23]:
tree_presence.head()

,County_norm,State_norm,tree_of_heaven
0,nan,al,1
1,autauga,al,1
2,bullock,al,1
3,butler,al,1
4,calhoun,al,1


In [24]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code,tree_of_heaven
0,01001,autauga,al,AL,3.0,1
1,01003,baldwin,al,AL,4.0,0
2,01005,barbour,al,AL,5.0,0
3,01007,bibb,al,AL,2.0,0
4,01009,blount,al,AL,2.0,0


In [25]:
base_counties['tree_of_heaven'].value_counts()

tree_of_heaven
0    2302
1     807
Name: count, dtype: int64

In [26]:
# ============================================================================
# STEP 7: CALCULATE COUNTY CENTROIDS (LAT/LON)
# ============================================================================
print("\n" + "=" * 80)
print("STEP 7: CALCULATING COUNTY CENTROIDS")
print("=" * 80)

# Filter counties shapefile to continental US
if 'STATEFP' in counties.columns:
    # State FIPS codes for continental US (01-56, excluding 02=AK, 15=HI)
    continental_state_fips = [f"{i:02d}" for i in range(1, 57) if i not in [2, 15]]
    counties = counties[counties['STATEFP'].isin(continental_state_fips)].copy()

# Calculate centroids
counties['centroid'] = counties.geometry.centroid
counties['centroid_lon'] = counties['centroid'].x
counties['centroid_lat'] = counties['centroid'].y

# Get FIPS code from shapefile
if 'GEOID' in counties.columns:
    counties['fips'] = counties['GEOID'].astype(str).str.zfill(5)
elif 'FIPS' in counties.columns:
    counties['fips'] = counties['FIPS'].astype(str).str.zfill(5)

centroids = counties[['fips', 'centroid_lat', 'centroid_lon']].copy()

print(f"Centroids calculated for {len(centroids)} counties")
print(centroids.head())

# Merge centroids with base counties
base_counties = base_counties.merge(centroids, on='fips', how='left')

print(f"Counties with centroids: {base_counties['centroid_lat'].notna().sum()}")
print(f"Counties missing centroids: {base_counties['centroid_lat'].isna().sum()}")



STEP 7: CALCULATING COUNTY CENTROIDS


/tmp/ipykernel_109515/1847122493.py:15: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  counties['centroid'] = counties.geometry.centroid


Centroids calculated for 3109 counties
    fips  centroid_lat  centroid_lon
0  40075     34.916384    -98.980921
1  46079     44.022044    -97.129376
2  37033     36.393274    -79.333523
3  48377     29.999803   -104.240522
4  39057     39.691522    -83.889861
Counties with centroids: 3099
Counties missing centroids: 10


In [27]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code,tree_of_heaven,centroid_lat,centroid_lon
0,01001,autauga,al,AL,3.0,1,32.534920,-86.642749
1,01003,baldwin,al,AL,4.0,0,30.660970,-87.749841
2,01005,barbour,al,AL,5.0,0,31.869603,-85.393197
3,01007,bibb,al,AL,2.0,0,32.998643,-87.126440
4,01009,blount,al,AL,2.0,0,33.980867,-86.567371


In [28]:
# ============================================================================
# STEP 8: DETERMINE FIRST INFESTATION YEAR PER COUNTY
# ============================================================================
print("\n" + "=" * 80)
print("STEP 8: DETERMINING FIRST INFESTATION YEAR")
print("=" * 80)

# Get the first year a county had a SLF report
first_infestation = report.groupby(['County_norm', 'State_norm'])['year'].min().reset_index()
first_infestation.rename(columns={'year': 'first_infestation_year'}, inplace=True)
first_infestation['State_norm'] = first_infestation['State_norm'].map(state_full_to_abbrev)



STEP 8: DETERMINING FIRST INFESTATION YEAR


In [29]:
first_infestation.head()

,County_norm,State_norm,first_infestation_year
0,adams,oh,2025
1,adams,pa,2021
2,albany,ny,2023
3,albemarle,va,2021
4,alexandria (independent city),va,2023


In [30]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code,tree_of_heaven,centroid_lat,centroid_lon
0,01001,autauga,al,AL,3.0,1,32.534920,-86.642749
1,01003,baldwin,al,AL,4.0,0,30.660970,-87.749841
2,01005,barbour,al,AL,5.0,0,31.869603,-85.393197
3,01007,bibb,al,AL,2.0,0,32.998643,-87.126440
4,01009,blount,al,AL,2.0,0,33.980867,-86.567371


In [31]:
report.head()

,objectid,ObsDate,year,Latitude,Longitude,County,State,Status_encoded,County_norm,State_norm
0,4999651,2017-08-10,2017,40.49447,-75.48765,lehigh,pennsylvania,1.0,lehigh,pennsylvania
1,7629151,2018-07-05,2018,40.39368,-76.03501,berks,pennsylvania,1.0,berks,pennsylvania
2,7629152,2018-07-05,2018,40.42861,-76.11261,berks,pennsylvania,1.0,berks,pennsylvania
3,7638053,2018-07-30,2018,40.48981,-76.10119,berks,pennsylvania,1.0,berks,pennsylvania
4,7811998,2018-09-21,2018,40.37313,-76.05336,berks,pennsylvania,1.0,berks,pennsylvania


In [32]:

print(f"Counties with at least one report: {len(first_infestation)}")
print(f"First infestation year range: {first_infestation['first_infestation_year'].min()} - {first_infestation['first_infestation_year'].max()}")
print(f"\nInfestation by year:")
print(first_infestation['first_infestation_year'].value_counts().sort_index())

# Merge with base counties
base_counties = base_counties.merge(
    first_infestation,
    on=['County_norm', 'State_norm'],
    how='left'
)


Counties with at least one report: 317
First infestation year range: 2014 - 2025

Infestation by year:
first_infestation_year
2014     1
2015     4
2016     1
2017     3
2018     8
2019    17
2020    45
2021    31
2022    69
2023    38
2024    38
2025    62
Name: count, dtype: int64


In [33]:
base_counties.head()

,fips,County_norm,State_norm,state_abbrev,urban_code,tree_of_heaven,centroid_lat,centroid_lon,first_infestation_year
0,01001,autauga,al,AL,3.0,1,32.534920,-86.642749,NaN
1,01003,baldwin,al,AL,4.0,0,30.660970,-87.749841,NaN
2,01005,barbour,al,AL,5.0,0,31.869603,-85.393197,NaN
3,01007,bibb,al,AL,2.0,0,32.998643,-87.126440,NaN
4,01009,blount,al,AL,2.0,0,33.980867,-86.567371,NaN


In [34]:


# ============================================================================
# STEP 9: CREATE COUNTY-YEAR COMBINATIONS
# ============================================================================
print("\n" + "=" * 80)
print("STEP 9: CREATING COUNTY-YEAR COMBINATIONS")
print("=" * 80)

# Determine year range (from data)
min_year = int(report['year'].min())
max_year = int(report['year'].max())
years = list(range(min_year, max_year + 1))

print(f"Year range: {min_year} - {max_year} ({len(years)} years)")

# Create all county-year combinations
county_year_list = []
for _, county_row in base_counties.iterrows():
    for year in years:
        county_year_list.append({
            'fips': county_row['fips'],
            'County_norm': county_row['County_norm'],
            'State_norm': county_row['State_norm'],
            'state_abbrev': county_row['state_abbrev'],
            'year': year,
            'urban_code': county_row['urban_code'],
            'tree_of_heaven': county_row['tree_of_heaven'],
            'centroid_lat': county_row['centroid_lat'],
            'centroid_lon': county_row['centroid_lon'],
            'first_infestation_year': county_row['first_infestation_year']
        })

df = pd.DataFrame(county_year_list)
print('df.head(): ', df.head())
print(f"Total county-year combinations: {len(df)}")
print(f"Unique counties: {df['fips'].nunique()}")
print(f"Years: {df['year'].nunique()}")



STEP 9: CREATING COUNTY-YEAR COMBINATIONS
Year range: 2014 - 2025 (12 years)
df.head():      fips County_norm State_norm state_abbrev  year  urban_code  \
0  01001     autauga         al           AL  2014         3.0   
1  01001     autauga         al           AL  2015         3.0   
2  01001     autauga         al           AL  2016         3.0   
3  01001     autauga         al           AL  2017         3.0   
4  01001     autauga         al           AL  2018         3.0   

   tree_of_heaven  centroid_lat  centroid_lon  first_infestation_year  
0               1      32.53492    -86.642749                     NaN  
1               1      32.53492    -86.642749                     NaN  
2               1      32.53492    -86.642749                     NaN  
3               1      32.53492    -86.642749                     NaN  
4               1      32.53492    -86.642749                     NaN  
Total county-year combinations: 37308
Unique counties: 3109
Years: 12


In [35]:
df.head()

,fips,County_norm,State_norm,state_abbrev,year,urban_code,tree_of_heaven,centroid_lat,centroid_lon,first_infestation_year
0,01001,autauga,al,AL,2014,3.0,1,32.53492,-86.642749,NaN
1,01001,autauga,al,AL,2015,3.0,1,32.53492,-86.642749,NaN
2,01001,autauga,al,AL,2016,3.0,1,32.53492,-86.642749,NaN
3,01001,autauga,al,AL,2017,3.0,1,32.53492,-86.642749,NaN
4,01001,autauga,al,AL,2018,3.0,1,32.53492,-86.642749,NaN


In [36]:

# ============================================================================
# STEP 10: CREATE INFESTED BINARY FEATURE
# ============================================================================
print("\n" + "=" * 80)
print("STEP 10: CREATING 'INFESTED' BINARY FEATURE")
print("=" * 80)

# A county is infested in year Y if first_infestation_year <= Y
# Once infested, remains infested for all subsequent years
df['infested'] = 0
df.loc[df['first_infestation_year'].notna() & 
       (df['year'] >= df['first_infestation_year']), 'infested'] = 1

print(f"Total infested county-years: {(df['infested'] == 1).sum()}")
print(f"Total non-infested county-years: {(df['infested'] == 0).sum()}")
print(f"\nInfested county-years by year:")
print(df.groupby('year')['infested'].sum().sort_index())



STEP 10: CREATING 'INFESTED' BINARY FEATURE
Total infested county-years: 1189
Total non-infested county-years: 36119

Infested county-years by year:
year
2014      1
2015      5
2016      6
2017      9
2018     17
2019     33
2020     78
2021    108
2022    177
2023    209
2024    243
2025    303
Name: infested, dtype: int64


In [37]:
df.head()

,fips,County_norm,State_norm,state_abbrev,year,urban_code,tree_of_heaven,centroid_lat,centroid_lon,first_infestation_year,infested
0,01001,autauga,al,AL,2014,3.0,1,32.53492,-86.642749,NaN,0
1,01001,autauga,al,AL,2015,3.0,1,32.53492,-86.642749,NaN,0
2,01001,autauga,al,AL,2016,3.0,1,32.53492,-86.642749,NaN,0
3,01001,autauga,al,AL,2017,3.0,1,32.53492,-86.642749,NaN,0
4,01001,autauga,al,AL,2018,3.0,1,32.53492,-86.642749,NaN,0


In [38]:
df['infested'].value_counts()
df['first_infestation_year'].value_counts()

first_infestation_year
2022.0    828
2025.0    720
2020.0    540
2024.0    408
2023.0    384
2021.0    360
2019.0    192
2018.0     96
2015.0     48
2017.0     36
2014.0     12
2016.0     12
Name: count, dtype: int64

In [39]:

# ============================================================================
# STEP 11: CREATE URBAN_EFFECTIVE FEATURE
# ============================================================================
print("\n" + "=" * 80)
print("STEP 11: CREATING 'URBAN_EFFECTIVE' FEATURE")
print("=" * 80)

# Urban effective = 1 if county is north of 38 degrees latitude
# Otherwise = 0 (meaning we don't use urban code for southern counties)
df['urban_effective'] = 0
df.loc[df['centroid_lat'] >= 38.0, 'urban_effective'] = 1

print(f"Counties with urban_effective = 1 (north of 38°): {df[df['urban_effective'] == 1]['fips'].nunique()}")
print(f"Counties with urban_effective = 0 (at/south of 38°): {df[df['urban_effective'] == 0]['fips'].nunique()}")



STEP 11: CREATING 'URBAN_EFFECTIVE' FEATURE
Counties with urban_effective = 1 (north of 38°): 1626
Counties with urban_effective = 0 (at/south of 38°): 1483


In [40]:
df.tail()

,fips,County_norm,State_norm,state_abbrev,year,urban_code,tree_of_heaven,centroid_lat,centroid_lon,first_infestation_year,infested,urban_effective
37303,56045,weston,wy,WY,2021,6.0,0,43.840485,-104.56783,NaN,0,1
37304,56045,weston,wy,WY,2022,6.0,0,43.840485,-104.56783,NaN,0,1
37305,56045,weston,wy,WY,2023,6.0,0,43.840485,-104.56783,NaN,0,1
37306,56045,weston,wy,WY,2024,6.0,0,43.840485,-104.56783,NaN,0,1
37307,56045,weston,wy,WY,2025,6.0,0,43.840485,-104.56783,NaN,0,1


In [41]:

# ============================================================================
# STEP 12: CREATE SPATIAL LAG FEATURES
# ============================================================================
print("\n" + "=" * 80)
print("STEP 12: CREATING SPATIAL LAG FEATURES")
print("=" * 80)

# Use the counties shapefile to find physically touching neighbors
print("Finding neighboring counties (physically touching)...")

# Ensure we have the FIPS column in the GeoDataFrame
if 'GEOID' in counties.columns:
    counties['fips'] = counties['GEOID'].astype(str).str.zfill(5)
elif 'FIPS' in counties.columns:
    counties['fips'] = counties['FIPS'].astype(str).str.zfill(5)

# Create a dictionary to store neighbors for each county
county_neighbors = {}

print(f"Total counties in shapefile: {len(counties)}")

# For each county, find all counties that touch it
for idx, county in counties.iterrows():
    fips_code = county['fips']
    county_geom = county.geometry
    
    # Find all counties whose geometry touches this county
    # Use touches() which returns True if boundaries touch
    neighbors = counties[counties.geometry.touches(county_geom)]['fips'].tolist()
    
    county_neighbors[fips_code] = neighbors

print(f"✓ Neighbor relationships calculated")

# Calculate statistics about neighbors
neighbor_counts = [len(v) for v in county_neighbors.values()]
print(f"Average neighbors per county: {np.mean(neighbor_counts):.1f}")
print(f"Min neighbors: {np.min(neighbor_counts)}")
print(f"Max neighbors: {np.max(neighbor_counts)}")
print(f"Median neighbors: {np.median(neighbor_counts):.0f}")

# Show sample of counties with their neighbors
print("\nSample of neighbor relationships:")
sample_fips = list(county_neighbors.keys())[:3]
for fips in sample_fips:
    county_info = counties[counties['fips'] == fips].iloc[0]
    county_name = county_info.get('NAME', 'Unknown')
    print(f"  {county_name} ({fips}): {len(county_neighbors[fips])} neighbors")

# Now calculate spatial lag for each county-year
print("\nCalculating spatial lag for each county-year...")

spatial_lag_features = []

for year in sorted(df['year'].unique()):
    print(f"  Processing year {year}...")
    year_data = df[df['year'] == year].copy()
    
    # Create a lookup for infested status by FIPS for this year
    infested_lookup = dict(zip(year_data['fips'], year_data['infested']))
    
    for fips_code in year_data['fips'].unique():
        neighbors = county_neighbors.get(fips_code, [])
        
        if len(neighbors) == 0:
            # No neighbors - spatial lag is 0 (island/isolated county)
            spatial_lag = 0.0
        else:
            # Count how many neighbors are infested
            neighbor_infested_count = sum([infested_lookup.get(n, 0) for n in neighbors])
            # Spatial lag = proportion of neighbors that are infested
            spatial_lag = neighbor_infested_count / len(neighbors)
        
        spatial_lag_features.append({
            'fips': fips_code,
            'year': year,
            'spatial_lag': spatial_lag
        })

spatial_lag_df = pd.DataFrame(spatial_lag_features)

print(f"\n✓ Spatial lag features calculated")
print(f"Spatial lag features shape: {spatial_lag_df.shape}")
print(f"\nSample spatial lag data:")
print(spatial_lag_df.head(10))

# Merge spatial lag features with main dataframe
df = df.merge(spatial_lag_df, on=['fips', 'year'], how='left')

print(f"\n✓ Spatial lag features merged")

# Summary statistics
print("\nSpatial Lag Summary Statistics:")
print(f"  Mean spatial lag: {df['spatial_lag'].mean():.3f}")
print(f"  Median spatial lag: {df['spatial_lag'].median():.3f}")
print(f"  Min spatial lag: {df['spatial_lag'].min():.3f}")
print(f"  Max spatial lag: {df['spatial_lag'].max():.3f}")
print(f"  Std spatial lag: {df['spatial_lag'].std():.3f}")

print("\nSpatial lag progression over years:")
print(df.groupby('year')['spatial_lag'].agg(['mean', 'median', 'max']).round(3))



STEP 12: CREATING SPATIAL LAG FEATURES
Finding neighboring counties (physically touching)...
Total counties in shapefile: 3109
✓ Neighbor relationships calculated
Average neighbors per county: 5.9
Min neighbors: 1
Max neighbors: 14
Median neighbors: 6

Sample of neighbor relationships:
  Kiowa (40075): 7 neighbors
  Lake (46079): 6 neighbors
  Caswell (37033): 7 neighbors

Calculating spatial lag for each county-year...
  Processing year 2014...
  Processing year 2015...
  Processing year 2016...
  Processing year 2017...
  Processing year 2018...
  Processing year 2019...
  Processing year 2020...
  Processing year 2021...
  Processing year 2022...
  Processing year 2023...
  Processing year 2024...
  Processing year 2025...

✓ Spatial lag features calculated
Spatial lag features shape: (37308, 3)

Sample spatial lag data:
    fips  year  spatial_lag
0  01001  2014          0.0
1  01003  2014          0.0
2  01005  2014          0.0
3  01007  2014          0.0
4  01009  2014         

In [42]:

# ============================================================================
# STEP 12: FINAL FEATURE SET
# ============================================================================
print("\n" + "=" * 80)
print("STEP 12: FINAL FEATURE SET")
print("=" * 80)

# Select final columns for ML
final_features = [
    'fips',
    'year',
    'centroid_lat',
    'centroid_lon',
    'infested',
    'tree_of_heaven',
    'urban_code',
    'urban_effective',
    'spatial_lag'
]

df_final = df[final_features].copy()

print(f"Final dataset shape: {df_final.shape}")
print(f"\nColumn data types:")
print(df_final.dtypes)
print(f"\nMissing values:")
print(df_final.isnull().sum())
print(f"\nFirst few rows:")
print(df_final.head(20))



STEP 12: FINAL FEATURE SET
Final dataset shape: (37308, 9)

Column data types:
fips                object
year                 int64
centroid_lat       float64
centroid_lon       float64
infested             int64
tree_of_heaven       int64
urban_code         float64
urban_effective      int64
spatial_lag        float64
dtype: object

Missing values:
fips                 0
year                 0
centroid_lat       120
centroid_lon       120
infested             0
tree_of_heaven       0
urban_code         120
urban_effective      0
spatial_lag          0
dtype: int64

First few rows:
     fips  year  centroid_lat  centroid_lon  infested  tree_of_heaven  \
0   01001  2014      32.53492    -86.642749         0               1   
1   01001  2015      32.53492    -86.642749         0               1   
2   01001  2016      32.53492    -86.642749         0               1   
3   01001  2017      32.53492    -86.642749         0               1   
4   01001  2018      32.53492    -86.642749 

In [43]:
df_final.head()

,fips,year,centroid_lat,centroid_lon,infested,tree_of_heaven,urban_code,urban_effective,spatial_lag
0,01001,2014,32.53492,-86.642749,0,1,3.0,0,0.0
1,01001,2015,32.53492,-86.642749,0,1,3.0,0,0.0
2,01001,2016,32.53492,-86.642749,0,1,3.0,0,0.0
3,01001,2017,32.53492,-86.642749,0,1,3.0,0,0.0
4,01001,2018,32.53492,-86.642749,0,1,3.0,0,0.0


In [44]:
df_final.to_csv('CS4824CapstoneData/features_grid.csv', index=False)
